In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pyfits as pyf
import os

In [3]:
def integrated_flux(SED, sigma, passband):
    mask = (passband[0, 0] <= SED[:, 0])&(SED[:, 0] <= passband[-1, 0])
    ipassband = interp(SED[mask, 0], passband[:, 0], passband[:, 1])
    
    weff = trapz(ipassband*SED[mask, 0], SED[mask, 0])/trapz(ipassband, SED[mask, 0])
    if (SED[mask, 1]>0.0).sum()*100./mask.sum()<99.0: return weff, -1.0, -1.0
    
    iflux = trapz(SED[mask, 0]*SED[mask, 1]*ipassband, SED[mask, 0])/\
            trapz(SED[mask, 0]*ipassband, SED[mask, 0])

    weights = SED[mask, 0]*ipassband/trapz(SED[mask, 0]*ipassband, SED[mask, 0])
    isigm = sqrt(trapz((weights*sigma[mask])**2, SED[mask, 0]))
    return weff, iflux, isigm

In [9]:
cdir = "../../Programs/challenge/"

jpas_list = sorted([os.path.join(root, file) \
             for root, subs, files in os.walk(cdir+"JPAS_filters/") \
             for file in files if file.endswith(".res")])
jplus_list = sorted([os.path.join(root, file) \
              for root, subs, files in os.walk(cdir+"JPLUS_filters/") \
              for file in files if file.endswith(".res")])

jpas_filters = [loadtxt(fname) for fname in jpas_list]
jplus_filters = [loadtxt(fname) for fname in jplus_list]

In [10]:
sed_list = ["spectro{0}.fits".format(i+1) for i in xrange(16)]
err_list = ["spectro{0}e.fits".format(i+1) for i in xrange(16)]

redshift = [0.580, 0.270, 0.325, 0.400, 0.405, 0.214, 0.520, 0.425, 0.480, 0.390, 0.250,
            0.315, 0.350, 0.770, 0.450, 0.490]

i = 0
for z, sedfile, errfile in zip(redshift, sed_list, err_list):
    s = pyf.open(sedfile)
    e = pyf.open(errfile)
    
    wl = array([s[0].header["CRVAL1"]+s[0].header["CDELT1"]*(1+j) \
                for j in xrange(s[0].header["NAXIS1"])])
    fl = s[0].data[0]
    sg = e[0].data[0]    
    sed = column_stack((wl, fl))
    
    with open("jpas_sed{0:02d}.txt".format(i+1), "w") as f:
        f.write("# redshift = {0:5.3f}\n".format(z))
        f.write("#\n")
        f.write("# {0:>14s}{1:>13s}{2:>13s}\n".format("filter", "flux", "sigma"))
        for j, filt in enumerate(jpas_filters):
            fname = jpas_list[j].rstrip(".res")
            fname = fname[fname.rfind("/")+1:]
            jpas_wl, jpas_fl, jpas_sg = integrated_flux(sed, sg, filt)
            f.write("  {0:>14s}{1:>13.4e}{2:>13.4e}\n".format(fname, jpas_fl, jpas_sg))

    with open("jplus_sed{0:02d}.txt".format(i+1), "w") as f:
        f.write("# redshift = {0:5.3f}\n".format(z))
        f.write("#\n")
        f.write("# {0:>7s}{1:>13s}{2:>13s}\n".format("filter", "flux", "sigma"))
        for j, filt in enumerate(jplus_filters):
            fname = jplus_list[j].rstrip(".res")
            fname = fname[fname.rfind("/")+1:]
            jplus_wl, jplus_fl, jplus_sg = integrated_flux(sed, sg, filt)
            f.write("  {0:>7s}{1:>13.4e}{2:>13.4e}\n".format(fname, jplus_fl, jplus_sg))
    i += 1